In [9]:
import datetime
from coinmetrics.api_client import CoinMetricsClient
import numpy as np
import pandas as pd
cm_key = 'F9wkf2BdTrwX885GQ2qF'
cm_client = CoinMetricsClient(cm_key)
from datetime import datetime, timezone

#### Import Data

In [10]:
cm_key = 'F9wkf2BdTrwX885GQ2qF'
cm_client = CoinMetricsClient(cm_key)

In [11]:
start = 1658440670000
mili_sec_timespan = 30000  # int(0.5 * 3600 * 1000)
end = start + mili_sec_timespan

market = "BTC_USD"
exchange = 'coinbase'

freq = '1m'
base = market.split('_')[0].lower()
quote = market.split('_')[1].lower()




In [12]:
def timestamp_to_strdate(value: int) -> str:
    converted = datetime.utcfromtimestamp(value/1000).strftime('%Y-%m-%dT%H:%M:%SZ')
    return converted


def time_to_timestamp(value):
    return datetime.timestamp(value) * 1000

def datetimeToTimestamp(value: datetime) -> float:
    return (value - datetime(1970, 1, 1)).total_seconds()

def timestamp_slippage(value: int) -> str:
    converted = datetime.utcfromtimestamp(value / 1000).strftime('%Y-%m-%d %H:%M:%S.%f+00:00')
    return converted

In [13]:
def get_candles(start,end,freq,exchange,market):
    candles = cm_client.get_market_candles(
        markets=f'{exchange}-{base}-{quote}-spot',
        frequency = freq,
        start_time=timestamp_to_strdate(start),
        end_time=timestamp_to_strdate(end),
        paging_from='start',
        page_size=10000)
        
    return candles.to_dataframe()

In [14]:
def call_cm_trades(start, end):
        time_start = timestamp_to_strdate(start)
        time_end = timestamp_to_strdate(end)
        trades = cm_client.get_market_trades(
            markets=f'{exchange}-{base}-{quote}-spot',
            start_time=time_start,
            end_time=time_end,
            paging_from='start',
            page_size=10000
        )
        trades_df = trades.to_dataframe()

        while len(trades_df) == 0:
            start = start - 1000
            end = end + 1000
            time_start = timestamp_to_strdate(start)
            time_end = timestamp_to_strdate(end)
            trades = cm_client.get_market_trades(
                markets=f'{exchange}-{base}-{quote}-spot',
                start_time=time_start,
                end_time=time_end,
                paging_from='start',
                page_size=10000
            )
            trades_df = trades.to_dataframe()

        return trades_df

def get_tick_data_cm(timestamp_start, timestamp_end):
        trades_df = call_cm_trades(timestamp_start, timestamp_end)
        trades_df['timestamps'] = [time_to_timestamp(trades_df['time'].iloc[i]) for i in range(len(trades_df))]
        trades_df['price'] = trades_df['price']
        trades_df['amount'] = trades_df['amount']
        trades_df['notional'] = trades_df['price'] * trades_df['amount']
        trades_df['side'] = trades_df['side']
        return trades_df

In [15]:
def get_slippage_amount(trades):
    ## the parameter "trades" comes from the function get_tick_data_cm()
    data_buys = {'timestamp': [], 'slippage': [], 'amount': [], 'notional': [], 'first_price': [], 'last_price': []}
    data_sells = {'timestamp': [], 'slippage': [], 'amount': [], 'notional': [], 'first_price': [], 'last_price': []}
    i = 0
    slippage_flag = False
    amount = 0
    notional = 0
    while i < len(trades) - 1:
        if trades['timestamps'].iloc[i + 1] == trades['timestamps'].iloc[i]:
            if not slippage_flag:
                slippage_flag = True
                first_price = trades['price'].iloc[i]
                amount += trades['amount'].iloc[i]
                notional += trades['amount'].iloc[i] * trades['price'].iloc[i]
            else:
                amount += trades['amount'].iloc[i]
                notional += trades['amount'].iloc[i] * trades['price'].iloc[i]
        elif slippage_flag:
            last_price = trades['price'].iloc[i]
            amount += trades['amount'].iloc[i]
            notional += trades['amount'].iloc[i] * trades['price'].iloc[i]
            try:
                if trades['side'].iloc[i] == 'buy':
                    data_buys['timestamp'].append(trades['timestamps'].iloc[i])
                    data_buys['slippage'].append(((last_price / first_price) - 1) * 100)
                    data_buys['amount'].append(amount)
                    data_buys['notional'].append(notional)
                    data_buys['first_price'].append(first_price)
                    data_buys['last_price'].append(last_price)
                elif trades['side'].iloc[i] == 'sell':
                    data_sells['timestamp'].append(trades['timestamps'].iloc[i])
                    data_sells['slippage'].append(((last_price / first_price) - 1) * 100)
                    data_sells['amount'].append(amount)
                    data_sells['notional'].append(notional)
                    data_sells['first_price'].append(first_price)
                    data_sells['last_price'].append(last_price)
            except:
                if (last_price - first_price) >= 0:  # If no side, then infer it.
                    data_buys['timestamp'].append(trades['timestamps'].iloc[i])
                    data_buys['slippage'].append(((last_price / first_price) - 1) * 100)
                    data_buys['amount'].append(amount)
                    data_buys['notional'].append(notional)
                    data_buys['first_price'].append(first_price)
                    data_buys['last_price'].append(last_price)
                else:
                    data_sells['timestamp'].append(trades['timestamps'].iloc[i])
                    data_sells['slippage'].append(((last_price / first_price) - 1) * 100)
                    data_sells['amount'].append(amount)
                    data_sells['notional'].append(notional)
                    data_sells['first_price'].append(first_price)
                    data_sells['last_price'].append(last_price)

            slippage_flag = False
            amount = 0
            notional = 0

        i += 1
    return data_buys, data_sells


In [16]:
cm_trades = call_cm_trades(start, end)

Error found for the query: 
 https://api.coinmetrics.io/v4/timeseries/market-trades?api_key=F9wkf2BdTrwX885GQ2qF&markets=coinbase-btc-usd-spot&page_size=10000&paging_from=start&start_time=2022-07-21T21%3A57%3A50Z&end_time=2022-07-21T21%3A58%3A20Z
Error details: {'type': 'wrong_credentials', 'message': 'Supplied credentials are not valid.'}


HTTPError: 401 Client Error: Unauthorized for url: https://api.coinmetrics.io/v4/timeseries/market-trades?api_key=F9wkf2BdTrwX885GQ2qF&markets=coinbase-btc-usd-spot&page_size=10000&paging_from=start&start_time=2022-07-21T21%3A57%3A50Z&end_time=2022-07-21T21%3A58%3A20Z

In [ ]:
cm_trades

In [ ]:
trades = get_tick_data_cm(start, end)

In [ ]:
trades

In [ ]:
slippage = get_slippage_amount(trades)

In [ ]:
slippage

#### Clean Data

In [ ]:
timestamp = trades["timestamps"]
call_cm_trades = cm_trades.join(timestamp)
notional = trades["notional"]
call_cm_trades = call_cm_trades.join(notional)
call_cm_trades

In [ ]:
slippageList = list(slippage)
slippageDict = slippageList[0]
df = pd.DataFrame.from_dict(slippageDict)
df

In [ ]:
print(call_cm_trades.to_string())

In [ ]:
df['datetime'] = df['timestamp'].apply(timestamp_slippage) 
df

In [ ]:
datetime = df['datetime']
datetime

In [ ]:
call_cm_trades['time2']=call_cm_trades['time'].dt.strftime('%Y-%m-%d %H:%M:%S.%f+00:00')
call_cm_trades["exists"] = call_cm_trades['time2'].isin(df['datetime'])
call_cm_trades2 = call_cm_trades[call_cm_trades.exists != False]
call_cm_trades2.reset_index(inplace = True) # Remove all index
call_cm_trades2['match'] = call_cm_trades2["time2"].shift() != call_cm_trades2["time2"]
call_cm_trades2 = call_cm_trades2[call_cm_trades2.match != False]
call_cm_trades2.reset_index(inplace = True)
call_cm_trades2['slippage']=df['amount']
call_cm_trades2

In [ ]:
call_cm_trades2['predict'] = 0


#### Get Training/Validation/Testing Sets

In [ ]:
call_cm_trades3 = call_cm_trades2.drop(['level_0','coin_metrics_id','index','market', 'time', 'database_time','side','timestamps','time2','match','exists'],axis=1)
call_cm_trades3

#### Create Testing Set

In [ ]:
X = call_cm_trades3.drop('slippage',axis=1)

In [ ]:
Y = call_cm_trades3['slippage']

In [ ]:
X

In [ ]:
Y

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=8)
X_train.shape, X_test.shape

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 1000, random_state = 0)
regressor.fit(X_train, y_train)

#### Prediction Analysis

In [ ]:
y_predict = regressor.predict(X_test)
df=pd.DataFrame({'Actual':y_test, 'Predicted':y_predict})
df

In [ ]:
# Find error. Source: https://machinelearningmastery.com/random-forest-for-time-series-forecasting/
error = abs(y_predict - y_test)
print('Mean Absolute Error:', round(np.mean(error), 4))

#Calculate mean absolute percentage error
MAPE = 100 * (error / y_test)
accuracy = 100 - np.mean(MAPE)
print('Accuracy:', round(accuracy, 4))

#### Analysis of Features

In [ ]:
#Bar plot of features from most important to least important using sklearn built-in method (percentage scale)
#https://www.yourdatateacher.com/2021/10/11/feature-selection-with-random-forest/

import matplotlib.pyplot as plt
f = list(zip(X,regressor.feature_importances_))
f.sort(key = lambda x : x[1])
plt.barh([x[0] for x in f_i],[x[1] for x in f_i])

plt.show()

In [ ]:
#Bar plot of features from most important to least important using permutation based feature importance (percentage scale)
#https://mljar.com/blog/feature-importance-in-random-forest/

from sklearn.inspection import permutation_importance
perm_importance = permutation_importance(regressor, X_test, y_test)

import matplotlib.pyplot as plt
f = list(zip(X,perm_importance))
f.sort(key = lambda x : x[1])
plt.barh([x[0] for x in f_i],[x[1] for x in f_i])

plt.show()

In [ ]:
#Feature importance using Shapley game theory (determined independent of model used)
#https://mljar.com/blog/feature-importance-in-random-forest/

import shap
explainer = shap.TreeExplainer(regressor)
shap_values = explainer.shap_values(X_test)

shap.summary_plot(shap_values, X_test, plot_type="bar")